# 0 Setting

In [1]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'
pretrained_model_name = 'model_microsoft.ckpt'

config = {
    'learning_rate': 4e-3,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,

    'n_epochs': 3000,
    'early_stop': 500,
    'device': device,
}


sector_id_list = [ # Not confirmed
    "XLK",  # Information Technology
    "XLV",  # Health Care
    "XLF",  # Financials
    "XLI",  # Industrials
    "XLY",  # Consumer Discretionary
    "XLE",  # Energy
    "XLB",  # Materials
    "XLC",  # Communication Services
    "XLU",  # Utilities
    "XLRE",  # Real Estate
    "XLP"  # Consumer Staples
]

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 2  #26

company_name = company_list[process_id]

config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_'+company_name+'.ckpt',
            # 'pretrained_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/premodel/' + pretrained_model_name,
            # 'continue_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_1.ckpt'
            }

feature = [
    # X_1
    'input_ids',
    'attention_mask',
    'section_dummy',
    'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 'logit0_bert',
    # 'logit1_bert',
    # 'logit0_finbert',
    # 'logit1_finbert',
    # 'logit2_finbert',

    # 'logit0_bert_average',
    # 'logit1_bert_average',
    # 'logit0_finbert_average',
    # 'logit1_finbert_average',
    # 'logit2_finbert_average',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',

    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

11


## (1) Import

In [2]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel, BertConfig

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Mounted at /content/drive


In [3]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [4]:
df = pd.read_csv(config_2['input_path'])
df.columns

FileNotFoundError: ignored

In [ ]:
df = df.sort_values(by='datetime', ascending=True)
df.shape

In [ ]:
# Only contain selected features
df = df[feature]
df.columns

## (2) check nan

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

## (2) Time Period Selection

In [ ]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

## (3) Transform str back to list

In [ ]:
# 将字符串转换回列表的函数
def string_to_list(s):
    return ast.literal_eval(s)

# 将列中的字符串转换回列表
df['input_ids'] = df['input_ids'].apply(string_to_list)
df['attention_mask'] = df['attention_mask'].apply(string_to_list)
df['section_dummy'] = df['section_dummy'].apply(string_to_list)
df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

In [ ]:
df['input_ids'][0][0]

In [ ]:
df['attention_mask'][0][0]

In [ ]:
df['section_dummy'][0][0]

## (3) List: Same amount of elements

In [ ]:
input_ids_list_length = len(df['input_ids'][0][0])
attention_mask_list_length = len(df['attention_mask'][0][0])
section_dummy_list_length = len(df['section_dummy'][0][0])
publication_dummy_list_length = len(df['publication_dummy'][0][0])

In [ ]:
# 找到最大的內部列表長度
max_inner_length = max(df['input_ids'].apply(len))

# 定義一個函數來填充內部列表，使其長度達到最大值
def pad_inner_list(lst, zero_list):
    while len(lst) < max_inner_length:
        lst.append(zero_list)  # 這裡可以填充任何你想要的值，例如 None

# 將 "input_ids" 列中的每個內部列表填充到最大長度
df['input_ids'].apply(pad_inner_list, zero_list=[0] * input_ids_list_length)
df['attention_mask'].apply(pad_inner_list, zero_list=[0] * attention_mask_list_length)
df['section_dummy'].apply(pad_inner_list, zero_list=[0] * section_dummy_list_length)
df['publication_dummy'].apply(pad_inner_list, zero_list=[0] * publication_dummy_list_length)
df

In [ ]:
df['section_dummy'][0]

In [ ]:
# 使用 apply 方法計算每個列表中元素的數量
# Note: This number is not about tokens. It's about the number of news in that day.
temp = df['input_ids'].apply(len)

# 打印 DataFrame
temp

In [ ]:
# 計算"input_ids"列中所有list的平均長度
average_length = df['input_ids'].apply(len).mean()

# 計算"input_ids"列中最長的list的長度
max_length = df['input_ids'].apply(len).max()

# 計算"input_ids"列中最短的list的長度
min_length = df['input_ids'].apply(len).min()

# 打印結果
print(f"平均長度: {average_length}")
print(f"最長長度: {max_length}")
print(f"最短長度: {min_length}")


## int to float (section, publication)

In [ ]:
type(df.input_ids[0][0])

In [ ]:
def recursive_convert_to_float(item):
    if isinstance(item, list):
        return [recursive_convert_to_float(x) if x is not None else None for x in item]
    else:
        return float(item) if item is not None else None

# 使用 apply 方法將函數應用於每個元素
df['section_dummy'] = df['section_dummy'].apply(recursive_convert_to_float)
df['publication_dummy'] = df['publication_dummy'].apply(recursive_convert_to_float)

In [ ]:
df['section_dummy'][0]

## (4) Train_test_split

In [ ]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']

In [ ]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

In [ ]:
# 2. train_test_split
# val dataset for final examination
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=config['seed'], shuffle=config['shuffle'])
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=config['seed'], shuffle=config['shuffle'])
X_train


## (5) Scaler

In [ ]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 'logit0_bert',
    # 'logit1_bert',
    # 'logit0_finbert',
    # 'logit1_finbert',
    # 'logit2_finbert',

    # 'logit0_bert_average',
    # 'logit1_bert_average',
    # 'logit0_finbert_average',
    # 'logit1_finbert_average',
    # 'logit2_finbert_average',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

## (6) Check number

In [ ]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


In [ ]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0], X_val.shape[0], X_test.shape[0])

# Model

## (1) Dataset & Dataloader

In [ ]:
# Dataset
X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_1
        self.input_ids = X['input_ids']
        self.attention_mask = X['attention_mask']

        # X_2
        self.X_2 = torch.tensor(X.drop(columns=X_1).values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.long)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_1
        input_ids_list = self.input_ids[idx : idx + self.seq_length].tolist() # All to list
        input_ids = torch.tensor(input_ids_list) # Then to tensor
        attention_mask_list = self.attention_mask[idx : idx + self.seq_length].tolist()
        attention_mask = torch.tensor(attention_mask_list)

        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return input_ids, attention_mask, X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [ ]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# Check loader output
for batch in train_loader:
    input_ids, attention_mask, X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    # print("X_2:", X_2)
    # print("Labels:", y)

    break


## (2) Model Architecture

### 0 Param setting

In [ ]:
new_config = {
    'h_text_size': 5,
    'h_c_size': 1,
}

config.update(new_config)

### 1 New design

In [ ]:
# New method: C as a scaler + activation function + layer normalization
class MyModel(nn.Module):
    def __init__(self, base_model, config, element_size, X_2_length, batch_size):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.batch_size = batch_size
        self.element_size = element_size
        self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                              0, 0, 0, 0, 0, 0, 0, 0], device=device)
        self.config = config

        # 1. News
        # text
        self.base_model = base_model
        self.fc1 = nn.Linear(768, config['h_text_size'])

        # 2. LSTM
        self.lstm_1 = nn.LSTM(config['h_text_size'] + X_2_length, 32, dropout=0.1, num_layers=2, batch_first=True, bidirectional=False)
        self.sequential = nn.Sequential(
            nn.Linear(32, 2)
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.batch_norm = nn.BatchNorm1d(config['h_text_size'])



    def forward(self, input_ids, attention_mask, X_2):
        # 1. BERT
        flattened_input_ids = input_ids.view(-1, 32)
        flattened_attention_mask = attention_mask.view(-1, 32)

        e_list = []
        for i in range(0, flattened_input_ids.size(0), self.element_size):
          sub_input_ids = flattened_input_ids[i:i+self.element_size]
          sub_attention_mask = flattened_attention_mask[i:i+self.element_size]

          non_zero_mask = (sub_input_ids != 0).any(dim=1)
          non_zero_input_ids = sub_input_ids[non_zero_mask]
          non_zero_attention_mask = sub_attention_mask[non_zero_mask]

          # input_ids, attention_mask
          out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
          out = out.pooler_output
          h_text = self.fc1(out)

          batch_norm_h = self.batch_norm(h_text)
          element_mean = torch.mean(batch_norm_h, dim=0)
          e_list.append(element_mean)

        temp_tensor = torch.stack(e_list)
        b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_text_size'])

        # After BERT

        h_tech = X_2

        h = torch.cat([b_tensor, h_tech], dim=2)

        # 3. LSTM
        out, _ = self.lstm_1(h)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        final_out = self.sequential(out)

        return final_out


## (4) Load Model

### 1. Load pretrain model

In [ ]:
# 載入預訓練模型
from transformers import AutoModelForSequenceClassification
# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))
# bert_config = BertConfig(hidden_dropout_prob=0.2)
model_name = 'ProsusAI/finbert'
# model_name = 'bert-base-uncased'
base_model = BertModel.from_pretrained(model_name)
# base_model = BertModel.from_pretrained(model_name, config=bert_config)
# base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Parameter
element_size = len(df['input_ids'][0])  # 114
X_2_length = len(feature) - 6


### 2. Initiate Model

In [ ]:
model = MyModel(base_model, config, element_size, X_2_length, config['batch_size'])

model.to(device)

### Extra: Contunue training

In [ ]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['save_path']))
# model.to(device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

## (5) Require_grad

In [ ]:
# Freeze all layers
for param in model.base_model.parameters():
  param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

for param in model.base_model.encoder.layer[11].parameters():
    param.requires_grad = True

# for param in model.base_model.bert.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.classifier.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

# Training

save based on acc and loss

In [ ]:
def trainer(model, train_loader, val_loader, config, device):

    criterion = config['criterion']
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_loss, best_acc, step, early_stop_count = config['n_epochs'], math.inf, 0, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for input_ids, attention_mask, X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        pred = model(input_ids, attention_mask, X_2)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      predicted_labels_list = []
      targets_list = []
      for input_ids, attention_mask, X_2, y in val_loader:
          input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(input_ids, attention_mask, X_2)
              _, predicted = torch.max(pred, 1)
              loss = criterion(pred, y)
              predicted_labels_list.extend(predicted.tolist())
              targets_list.extend(y.tolist())
              loss_record.append(loss.item())
      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if (accuracy > best_acc) or ((accuracy == best_acc) and (mean_valid_loss < best_loss)):
          best_acc = accuracy
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_acc))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return




Old trainer

In [ ]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']

#     # ----------------------Learning Rate-----------------------
#     # optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
#     optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

#     # learning_rates = {
#     #     'base_model': 1e-5,  # 设置base_model的学习率
#     #     'base_model_fc': 1e-5,
#     #     'other_params': 1e-3  # 设置其他参数的学习率
#     # }

#     # param_groups = [
#     #     {'params': model.base_model.parameters(), 'lr': learning_rates['base_model']},
#     #     {'params': model.fc1.weight, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc1.bias, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc2.weight, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc2.bias, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc_h_c.weight, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc_h_c.bias, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc_h_news.weight, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc_h_news.bias, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.lstm_1.weight_ih_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_hh_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_ih_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_hh_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_ih_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_hh_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_ih_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_hh_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.sequential[0].weight, 'lr': learning_rates['other_params']},
#     #     {'params': model.sequential[0].bias, 'lr': learning_rates['other_params']},
#     # ]
#     # optimizer = torch.optim.AdamW(param_groups)
#     # ----------------------------------------------

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#       for input_ids, attention_mask, section, publication, X_2, y in train_loader:
#         optimizer.zero_grad()  # Set gradient to zero

#         # Forward pass
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         pred = model(input_ids, attention_mask, section, publication, X_2)
#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       # 2. Evaluation
#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       predicted_labels_list = []
#       targets_list = []
#       for input_ids, attention_mask, section, publication, X_2, y in val_loader:
#           input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(input_ids, attention_mask, section, publication, X_2)
#               _, predicted = torch.max(pred, 1)
#               loss = criterion(pred, y)
#               predicted_labels_list.extend(predicted.tolist())
#               targets_list.extend(y.tolist())
#               loss_record.append(loss.item())
#       accuracy = accuracy_score(targets_list, predicted_labels_list)

#       # Mean
#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       # 3. Judge of saving model
#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


In [ ]:
# 全部訓練
trainer(model, train_loader, val_loader, config, device)


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Evaluate

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

model = MyModel(base_model, config, element_size, X_2_length, batch_size=1)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()

accuracy_list = []

# 1. Train part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_train_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))


# 2. Val part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_val_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))

# 3. Test part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_test_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)